In [1]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = '/home/data/train.json'

# Read the JSON file
with open(train_path, 'r') as f:
    train_data = json.load(f)

print(f"Number of records: {len(train_data)}")
print(f"Type of data: {type(train_data)}")
print(f"First record keys: {list(train_data[0].keys())}")

Number of records: 2878
Type of data: <class 'list'>
First record keys: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_re

In [2]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(train_data)
print(f"DataFrame shape: {df.shape}")
print("\nColumn names:")
for col in df.columns:
    print(f"- {col}")

DataFrame shape: (2878, 32)

Column names:
- giver_username_if_known
- number_of_downvotes_of_request_at_retrieval
- number_of_upvotes_of_request_at_retrieval
- post_was_edited
- request_id
- request_number_of_comments_at_retrieval
- request_text
- request_text_edit_aware
- request_title
- requester_account_age_in_days_at_request
- requester_account_age_in_days_at_retrieval
- requester_days_since_first_post_on_raop_at_request
- requester_days_since_first_post_on_raop_at_retrieval
- requester_number_of_comments_at_request
- requester_number_of_comments_at_retrieval
- requester_number_of_comments_in_raop_at_request
- requester_number_of_comments_in_raop_at_retrieval
- requester_number_of_posts_at_request
- requester_number_of_posts_at_retrieval
- requester_number_of_posts_on_raop_at_request
- requester_number_of_posts_on_raop_at_retrieval
- requester_number_of_subreddits_at_request
- requester_received_pizza
- requester_subreddits_at_request
- requester_upvotes_minus_downvotes_at_request

In [ ]:
# Analyze target distribution
target = 'requester_received_pizza'
print("Target distribution:")
print(df[target].value_counts())
print(f"\nTarget percentages:")
print(df[target].value_counts(normalize=True) * 100)
print(f"\nClass imbalance ratio: {df[target].value_counts().min() / df[target].value_counts().max():.4f}")